# Objectives of Data cleaning 
The Purpose of this notebook is to: 
- convert Raw Data into Analysis-ready Data
- remove Data Leakage 
- ensure Data Integrity

In [23]:
import numpy as np
import pandas as pd

In [24]:
# importing Dataset 
df = pd.read_csv("../data/BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


## Removing Data Leakage Columns
- naive bayes colums are model output
- these are not available in in real-world banking 
- we'll not use them in risk analysis 

In [25]:
# removing naive bayes columns
df.drop(columns=[
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
], inplace=True, errors='ignore')

df.to_csv("cleaned_bank_data.csv", index=False)


In [26]:
# cheking shape again to confirm
df.shape

(10127, 21)

## Identifier Handling
LIENTNUM represents a unique customer identifier.
It is used to distinguish one customer record from another.
This variable does not contain any behavioral, financial, or demographic information that contributes to risk assessment.

Therefore:
- It will not be used in analytical calculations or risk signal detection.
- It may be retained only for record tracking or customer referencing purposes.
- It will be excluded from any statistical or segmentation analysis.

Proper handling of identifier variables ensures analytical integrity and prevents meaningless variables from influencing insights.

## Check Again for Missing Values & Duplicates
checking again for missing & duplicate values after data cleaning is a good practice 

In [27]:
# checking for missing values
df.isnull().sum()

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

In [28]:
# cheking for duplicates 
df.duplicated().sum()

np.int64(0)

## Data Type Consistency
- Numerical variables (e.g., Credit_Limit, Total_Trans_Amt, Avg_Utilization_Ratio) should be stored as numeric data types to allow statistical analysis.
- Categorical variables (e.g., Gender, Education_Level, Income_Category) should be stored as object or category types.
- The target variable Attrition_Flag must remain categorical since it represents customer status.
- Identifier column CLIENTNUM is numeric but functions as an identifier, not as a measurable quantity.

Ensuring correct data types is critical because incorrect types can lead to faulty analysis, incorrect aggregations, or misleading insights.

At this stage, no type conversion is required as the dataset appears structurally consistent.

## Logical Consistency Checks

In [29]:
print("Checking for negative values in financial columns...")

financial_cols = [
    "Credit_Limit",
    "Total_Revolving_Bal",
    "Total_Trans_Amt",
    "Total_Trans_Ct"
]

for col in financial_cols:
    if col in df.columns:
        print(f"{col} - Minimum Value: {df[col].min()}")


print("Checking Utilization Ratio Range...")

if "Avg_Utilization_Ratio" in df.columns:
    print("Min Utilization:", df["Avg_Utilization_Ratio"].min())
    print("Max Utilization:", df["Avg_Utilization_Ratio"].max())

print("\nChecking Months-Based Columns...\n")

month_cols = [
    "Months_on_book",
    "Months_Inactive_12_mon",
    "Contacts_Count_12_mon"
]

for col in month_cols:
    if col in df.columns:
        print(f"{col} - Min: {df[col].min()}, Max: {df[col].max()}")

print("\nLogical consistency check completed.")


Checking for negative values in financial columns...
Credit_Limit - Minimum Value: 1438.3
Total_Revolving_Bal - Minimum Value: 0
Total_Trans_Amt - Minimum Value: 510
Total_Trans_Ct - Minimum Value: 10
Checking Utilization Ratio Range...
Min Utilization: 0.0
Max Utilization: 0.999

Checking Months-Based Columns...

Months_on_book - Min: 13, Max: 56
Months_Inactive_12_mon - Min: 0, Max: 6
Contacts_Count_12_mon - Min: 0, Max: 6

Logical consistency check completed.


## Final Dataset Validation


In [30]:
print("Final Dataset Shape:", df.shape)
print("\nFinal Dataset Info:\n")

df.info()

print("\nDuplicate Records:", df.duplicated().sum())

print("\nMissing Values:\n")
print(df.isnull().sum())

Final Dataset Shape: (10127, 21)

Final Dataset Info:

<class 'pandas.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  str    
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  str    
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  str    
 6   Marital_Status            10127 non-null  str    
 7   Income_Category           10127 non-null  str    
 8   Card_Category             10127 non-null  str    
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit     

## Data Cleaning Summary
The following cleaning and validation steps were completed:

Removed data leakage columns (model probability outputs).

Handled unique identifier (CLIENTNUM excluded from analysis).

Confirmed absence of missing values.

Verified no duplicate customer records.

Conducted logical consistency checks on financial and behavioral variables.

Validated realistic value ranges using business reasoning.

The dataset is now structurally clean, logically consistent, and ready for feature engineering and advanced analysis.Tm